In [1]:
import openai
import time
import pandas as pd
import tqdm
import os

# Function to get an integer response from OpenAI API
def get_integer_from_openai(prompt):
    openai.api_key = os.getenv("OPENAI_KEY")
    
    max_retries = 10
    retries = 0
    while retries < max_retries:
        try:
            # Call the OpenAI API
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )
            # Extract the text response from the API
            response_text = int(response.choices[0].message.content)
            break
        except Exception as e:
            retries += 1
            print('maxing out retries for the following error: ' + str(e))
            time.sleep(1)

    return response_text


def get_jsonl(df):
                
    jsonlist = []
    for index, row in df.iterrows():
        jsonlist.append({
        "custom_id": str(row['No']),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model":"gpt-4o",
            "messages": [
                {
                    "role": "system",
                    "content": " "
                },
                {
                    "role": "user",
                    "content": row['prompt']
                }
            ],
            "max_tokens": 1000,
            "tools": [
                        {
                            "type": "function",
                            "function": {
                                "name": "task1",
                                "description": " ",
                                    "parameters":{
                                    "type":"object",
                                    "properties":{      
                                        "company_matching":{
                                            "type":"string",
                                            "description":"classify company matching into yes: same company or subsidiary of one another, or no",
                                            "enum":["yes", "no"]
                                        }     
                                        },
                                    "required":["company_matching"]
                                                }
                            },
                        }
                    ],
            "tool_choice": {"type": "function", "function": {"name": "task1"}}
        }
    })
    return jsonlist


In [18]:


loan_tables = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Firm all.xlsx', 'rb'),
            sheet_name='Sheet1').dropna(subset=['firm1', 'firm2']).drop(columns=['使用内置搜索综合判断firm1和firm2是同一个公司或者有母子公司关系吗']).head(100)
print(loan_tables.head())

res = []
for _, row in tqdm.tqdm(loan_tables.iterrows()):
    print(row['firm1'], row['firm2'])

    prompt = f"""
    You are a stock analyst, the task is to decide whether two firm names represent the same company or is subsidiary of the other.
    The first firm name is {row['firm1']}, the second firm name is {row['firm2']}.

    Use not only the name similarity but also the context and history of the company. For example, Merrill Lynch and Bank of America are the same company or subsidiary of the other, while Merrill Lynch and Goldman Sachs are not.

    Try your best to make a decision with representing or not representing the same company. 
    return 0 if they are not the same company or subsidiary of the other, return 1 if they represent same company or subsidiary of the other. return ONLY the indicated integer and NOTHING ELSE.
    """
    
    result = get_integer_from_openai(prompt)
    print(result)

    res.append(result)

   No  ID_MS                    firm1                         firm2   ID_AS
0   1  15294          swiss bank corp           swiss national bank  202552
1   2  49619            chemical bank          chemical tankers inc  322023
2   3  50202         bank of new york  bank of new york mellon corp    2019
3   4  12677         bank of montreal              bank of montreal   15580
4   5  92873  detroit bank & trust co             detroit edison co   65089


0it [00:00, ?it/s]

swiss bank corp swiss national bank


1it [00:01,  1.07s/it]

0
chemical bank chemical tankers inc


...

0
hypobank international sa cm international sa


99it [01:32,  1.10it/s]

0
united virginia bank united bank ltd


100it [01:34,  1.06it/s]

0


In [19]:
loan_tables['result'] = res
loan_tables.to_csv('loan_tble_matching_result.csv', index=False)

In [26]:
# prepare batch file jsonl
loan_tables = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Firm all.xlsx', 'rb'),
            sheet_name='Sheet1').dropna(subset=['firm1', 'firm2']).drop(columns=['使用内置搜索综合判断firm1和firm2是同一个公司或者有母子公司关系吗'])
print(loan_tables.head())

def add_prompt(row):
    return f"""You are a stock analyst, the task is to decide whether two firm names represent the same company or is subsidiary of the other.
    The first firm name is {row['firm1']}, the second firm name is {row['firm2']}.

    Use not only the name similarity but also the context and history of the company. For example, Merrill Lynch and Bank of America are the same company or subsidiary of the other, while Merrill Lynch and Goldman Sachs are not.

    return 'no' if they are not the same company or subsidiary of the other, return 'yes' if they represent same company or subsidiary of the other. return ONLY the indicated integer and NOTHING ELSE.
    """
loan_tables['prompt'] = loan_tables.apply(lambda row: add_prompt(row), axis=1)
print(loan_tables)

import json
res = get_jsonl(loan_tables.iloc[:30000])
# Write data to JSONL file
with open('loan_jsonl_1.jsonl', 'w') as file:
    for user in res:
        file.write(json.dumps(user) + '\n')

res = get_jsonl(loan_tables.iloc[30000:60000])
# Write data to JSONL file
with open('loan_jsonl_2.jsonl', 'w') as file:
    for user in res:
        file.write(json.dumps(user) + '\n')

res = get_jsonl(loan_tables.iloc[60000:])
# Write data to JSONL file
with open('loan_jsonl_3.jsonl', 'w') as file:
    for user in res:
        file.write(json.dumps(user) + '\n')


   No  ID_MS                    firm1                         firm2   ID_AS
0   1  15294          swiss bank corp           swiss national bank  202552
1   2  49619            chemical bank          chemical tankers inc  322023
2   3  50202         bank of new york  bank of new york mellon corp    2019
3   4  12677         bank of montreal              bank of montreal   15580
4   5  92873  detroit bank & trust co             detroit edison co   65089
          No   ID_MS                                       firm1  \
0          1   15294                             swiss bank corp   
1          2   49619                               chemical bank   
2          3   50202                            bank of new york   
3          4   12677                            bank of montreal   
4          5   92873                     detroit bank & trust co   
...      ...     ...                                         ...   
93662  93663  177598  safeguard world international holdings ltd   


In [7]:
# load in batched output 
import json
# import the file back row by row
res_data = []
with open('/Users/zhenggong/Documents/GitHub/IBES_selected/runnables/batch_jFQNkXpSipDZbkqXn6iCvPYC_output.jsonl', 'r') as f:
    for line in f:

        json_obj = json.loads(line)
        response = json.loads(json_obj['response']['body']['choices'][0]['message']['tool_calls'][0]['function']['arguments'])

        rowid = json_obj['custom_id']

        data = {'No': rowid,
            'company_matching': response['company_matching'],
                'model': json_obj['response']['body']['model'],
                }
        res_data.append(data)

with open('/Users/zhenggong/Documents/GitHub/IBES_selected/runnables/batch_8lBh8IBHZFCaY6xbyVp1HeyO_output.jsonl', 'r') as f:
    for line in f:

        json_obj = json.loads(line)
        response = json.loads(json_obj['response']['body']['choices'][0]['message']['tool_calls'][0]['function']['arguments'])

        rowid = json_obj['custom_id']

        data = {'No': rowid,
            'company_matching': response['company_matching'],
                'model': json_obj['response']['body']['model'],
                }
        res_data.append(data)


with open('/Users/zhenggong/Documents/GitHub/IBES_selected/runnables/batch_cJZPLtO8BMCvmC7iVBzWPzf3_output.jsonl', 'r') as f:
    for line in f:

        json_obj = json.loads(line)
        response = json.loads(json_obj['response']['body']['choices'][0]['message']['tool_calls'][0]['function']['arguments'])

        rowid = json_obj['custom_id']

        data = {'No': rowid,
            'company_matching': response['company_matching'],
                'model': json_obj['response']['body']['model'],
                }
        res_data.append(data)

res_data = pd.DataFrame(res_data)
res_data['No'] = res_data['No'].astype(int)

loan_tables = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Firm all.xlsx', 'rb'),
            sheet_name='Sheet1').dropna(subset=['firm1', 'firm2']).drop(columns=['使用内置搜索综合判断firm1和firm2是同一个公司或者有母子公司关系吗'])
loan_tables['No'] = loan_tables['No'].astype(int)

res_data = pd.merge(res_data, loan_tables, left_on=['No'], right_on=['No'], how='left')
print(res_data)
res_data.to_csv('loan_tble_matching_result.csv', index=False)

          No company_matching              model   ID_MS  \
0          1               no  gpt-4o-2024-05-13   15294   
1          2               no  gpt-4o-2024-05-13   49619   
2          3              yes  gpt-4o-2024-05-13   50202   
3          4              yes  gpt-4o-2024-05-13   12677   
4          5               no  gpt-4o-2024-05-13   92873   
...      ...              ...                ...     ...   
93662  59996              yes  gpt-4o-2024-05-13   22604   
93663  59997               no  gpt-4o-2024-05-13   27118   
93664  59998              yes  gpt-4o-2024-05-13   13947   
93665  59999              yes  gpt-4o-2024-05-13  102132   
93666  60000               no  gpt-4o-2024-05-13  102088   

                                firm1                         firm2   ID_AS  
0                     swiss bank corp           swiss national bank  202552  
1                       chemical bank          chemical tankers inc  322023  
2                    bank of new york  bank o